In [1]:
import pandas as pd 
import os 
import numpy as np 
import sys 
import re
import json 
from helpers import match_city_with_filters, get_city_relevance_score, get_improvement_percentage
import ast

In [2]:
data_dir = "/".join(os.getcwd().split("/")[:-2]) + "/data/conv-trs/"
results_dir = data_dir + "multi-agent/results/"
print(results_dir)

/Users/adithisatish/TUM/Academics/thesis/travel-crs/data/conv-trs/multi-agent/results/


In [3]:
listings_df = pd.read_csv(f"{data_dir}kg-generation/new-kg/data/merged_listing.csv")

In [4]:
samples_df = pd.read_csv(f"{data_dir}multi-agent/sample-data/llama3point2_sample.csv")

In [5]:
print(samples_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  45 non-null     object
 1   config     45 non-null     object
 2   city       45 non-null     object
 3   query_v    45 non-null     object
dtypes: object(4)
memory usage: 1.5+ KB
None


In [6]:
sample_configs = pd.concat([
    samples_df[samples_df['config_id'].str.contains("pop_low")].apply(lambda x: x.sample(5, random_state=42)), 
    samples_df[samples_df['config_id'].str.contains("pop_medium")].apply(lambda x: x.sample(5, random_state=42)), 
    samples_df[samples_df['config_id'].str.contains("pop_high")].apply(lambda x: x.sample(5, random_state=42)), 
])

print(sample_configs.info())

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 24 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  15 non-null     object
 1   config     15 non-null     object
 2   city       15 non-null     object
 3   query_v    15 non-null     object
dtypes: object(4)
memory usage: 600.0+ bytes
None


In [7]:
sample_configs['config'] = sample_configs['config'].apply(ast.literal_eval)

In [ ]:
mami_results = [] 
mami_dir = f"{results_dir}prompt_ranking_aggressive/gemini2point5flash/states/"
rounds=10

improvement = []
for file in os.listdir(mami_dir):
    config_id = (file.split("states_")[1]).split(".json")[0]

    # if config_id not in sample_configs['config_id'].tolist():
    #     continue
    
    with open(f"{mami_dir}{file}", "r") as f: 
        data = json.load(f)

    # filters = sample_configs[sample_configs['config_id'] == config_id]['config'].values[0]['filters']
    filters = ast.literal_eval(samples_df[samples_df['config_id'] == config_id]['config'].values[0])['filters']
    # print(filters)
    # print(get_improvement_percentage(data, filters, kb))
    improvement.append(get_improvement_percentage(data, filters, listings_df))

[{'round': 4, 'improvement': 50.00000000000001}, {'round': 0, 'improvement': 0}, {'round': 3, 'improvement': 99.99999999999999}, {'round': 2, 'improvement': 50.00000000000001}, {'round': 8, 'improvement': 50.00000000000001}, {'round': 1, 'improvement': 33.333333333333336}, {'round': 9, 'improvement': 200.00000000000003}, {'round': 0, 'improvement': 0}, {'round': 7, 'improvement': 50.00000000000001}, {'round': 0, 'improvement': 0}, {'round': 6, 'improvement': 33.333333333333336}, {'round': 1, 'improvement': 33.333333333333336}, {'round': 0, 'improvement': 0}, {'round': 0, 'improvement': 0}, {'round': 0, 'improvement': 0}, {'round': 3, 'improvement': 33.333333333333336}, {'round': 6, 'improvement': 300.0}, {'round': 0, 'improvement': 0}, {'round': 0, 'improvement': 0}, {'round': 6, 'improvement': 33.333333333333336}, {'round': 0, 'improvement': 0}, {'round': 0, 'improvement': 0}, {'round': 5, 'improvement': 33.333333333333336}, {'round': 5, 'improvement': 100.0}, {'round': 1, 'improvemen

In [9]:
print(f"Average Improvement shown: {round(np.mean(list(map(lambda x: x['improvement'], improvement))), 2)}%")

Average Improvement shown: 48.52%


In [13]:
print(f"Average Improvement shown: {round(np.median(list(map(lambda x: x['round'], improvement))), 2)}%")

Average Improvement shown: 1.0%
